In [40]:
import os
import pathlib
import wave

import glob
import random

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [41]:
# shuffle filenames

data_dir = "recordings"

filenames = glob.glob(str(data_dir) + "/*")
random.shuffle(filenames)
num_samples = len(filenames)

print(num_samples)
print(filenames[:5])

3000
['recordings/7_yweweler_6.wav', 'recordings/5_george_9.wav', 'recordings/6_yweweler_48.wav', 'recordings/3_george_11.wav', 'recordings/8_nicolas_31.wav']


In [42]:
tenth = int(num_samples * 0.1)
eightyth = tenth * 8

train_files = filenames[:eightyth]
val_files = filenames[eightyth: eightyth + tenth]
test_files = filenames[eightyth + tenth:]

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))

Training set size 2400
Validation set size 300
Test set size 300


In [43]:
def decode_audio(file_path):
    # Read file to get buffer                                                                                               
    ifile = wave.open(file_path)
    samples = ifile.getnframes()
    audio = ifile.readframes(samples)

    # Convert buffer to float32 using NumPy                                                                                 
    audio_as_np_int16 = np.frombuffer(audio, dtype=np.int16)
    audio_as_np_float32 = audio_as_np_int16.astype(np.float32)

    # Normalise float32 array so that values are between -1.0 and +1.0                                                      
    max_int16 = 2**15
    audio_normalised = audio_as_np_float32 / max_int16
    
    return audio_normalised

def get_label(file_path):
    parts = file_path.split("/")
    
    label = int(parts[1].split("_")[0])
    
    # Note: You'll use indexing here instead of tuple unpacking to enable this
    # to work in a TensorFlow graph.
    return label

def get_waveform_and_label(file_path):
    label = get_label(file_path)
#     audio_binary = tf.io.read_file(file_path)
    waveform = decode_audio(file_path)
#     return waveform, label
    return waveform, label

In [44]:
labeled = []

for file_path in train_files:
    datapoint = get_waveform_and_label(file_path)
    print(datapoint)
    labeled.append(datapoint)
    break

    
l2 = np.array(labeled)
tf.data.Dataset(l2)

(array([ 0.0000000e+00,  9.1552734e-05,  1.2207031e-04, ...,
       -3.9367676e-03, -3.8452148e-03, -3.1127930e-03], dtype=float32), 7)


/Users/jasminewu/Documents/6115/II/Spoken_Digits/venv/lib/python3.7/site-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


TypeError: Can't instantiate abstract class DatasetV2 with abstract methods _inputs, element_spec